# Intermediate Python
### Patrick Loeber, python-engineer.com
### https://www.youtube.com/watch?v=HGOBQPFzWKo
(2:20:10)
September 16, 2022

## LOGGING:
Looking at the different logging levels, configuration options, how to log in different modules, how to use different log handlers, how to capture stack traces in your log, and how to use rotating file handler

#### The different logging levels are DEBUG, INFO, WARNING, ERROR, CRITICAL:
* logging.debug ('This is a debug message')
* logging.info ('This is an info message')
* logging.warning ('This is a warning message')
* logging.error ('This is an error message')
* logging.critical ('This is a critical message')

These indicate the severity of the events.
By default, only errors with levels of warning or higher ar printed. This can be changed by setting the basic configuration, usually done right after importing logging.<br>
<br>
#### The format for logging.basicConfig() is:
logging.basicConfig(level=LOGGING_LEVEL, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s, datefmt='%m/%d/%Y %I:%M:%S %p')')
* The asctime is the time the message was logged.
* The name is the name of the logger.
* The levelname is the level of the message.
* The message is the message you want to log.
* datefmt is an optional argument that can be used to specify the format of the date.


In [14]:
import logging
# Set level, format, and date format
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - '
                                                '%(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')

#### For logging in your own modules, start a helpers file, and run the following:

In [18]:
# Creates a logger with the name of the module it is input inside
logger = logging.getLogger(__name__)
logger.propagate = False    # will not propegate to base logger
logger.info('Hello, from Loggy, the logger!')

#### Then import the module where the above logger code has been input into your main file / module

#### Logger setup:
* logger.propegation is a boolean that determines if the logger will pass the message on to the parent logger.
* logger.disabled is a boolean that determines if the logger will ignore all messages.
* logger.level is an integer that determines the lowest severity message that will be passed to the logger.
* logger.name is a string that is the name of the logger.
* logger.parent is a reference to the parent logger.
* logger.handlers is a list of the handlers that are attached to the logger.


#### Log Handing / Handler Objects: are responsible for dispatching the appropriate log message to the handler's specific destination. You can use different handlers to send log messages standard output stream or to files, emails, etc.

Logging handlers are used to send the log messages to the appropriate destination.
The different handlers are:
* logging.StreamHandler()
* logging.FileHandler(filename)
* logging.handlers.RotatingFileHandler(filename, maxBytes, backupCount)
* logging.handlers.TimedRotatingFileHandler(filename, when, interval, backupCount)
* logging.handlers.HTTPHandler(host, url, method)
* logging.handlers.SMTPHandler(mailhost, fromaddr, toaddrs, subject)
* logging.handlers.SysLogHandler(address)
* logging.NullHandler()

In [22]:
# Setting a log handler
logger = logging.getLogger(__name__)

# create handler
stream_handler = logging.StreamHandler()
file_handler = logging.FileHandler("file.log")

# setting level and format for handlers
stream_handler.setLevel(logging.WARNING)
file_handler.setLevel(logging.ERROR)

# specify a formatter
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')

# Set the formatter
stream_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# hand the handler to the logger
logger.addHandler(stream_handler)
logger.addHandler(file_handler)

# set logger messages
logger.warning("This is a warning... Now, you be good! ")
logger.error("This is an error. :( ")

# Now we have a log file in the same directory as this file, which
# will be logged to.

__main__ - WARNING - This is a warning... Now, you be good! 
__main__ - WARNING - This is a warning... Now, you be good! 
__main__ - ERROR - This is an error. :( 
__main__ - ERROR - This is an error. :( 


### OTHER CONFIGURATIONS: There is also DictConfiguration, etc (see documentation)

In [25]:
# A logging configuration file can be used instead of the above
# method
# The following would be the logging.conf file
# In the main file, the following lines would be used
# import logging.config
# logging.config.fileConfig('logging.conf')

# Specifying the name of the logger
[loggers]
keys = root, simpleExample

# Specifying the handlers
[handlers]
keys = consoleHandler, fileHandler, stream_handler

# Specifying the formatter
[formatters]
keys = simpleFormatter

# Assigning the arguments to the loggers
[logger_root]
level = DEBUG
handlers = consoleHandler

[logger_simpleExample]
level = INFO
handlers = consolehandler
qualname = simpleExample
propagate = 0

[handler_consoleHandler]
class = streamHandler
level = DEBUG
formatter = simpleFormatter
args = ('sys.stdout',)

[formatter_]simpleFormatter
format = %(asctime)s - %(name)s - %(levelname)s - %(message)s
datefmt =



SyntaxError: invalid syntax (3245434057.py, line 32)

### CAPTURING STACK TRACES in logs:
Useful when code raises an exception.

In [27]:
try:
    a = [1, 2, 3]
    val = a[4]
except IndexError as e:
    logging.error(e, exc_info = True)

2022-09-17 09:10:17,924 - root - ERROR - list index out of range
Traceback (most recent call last):
  File "/var/folders/wx/x69ggs7s5cgd2gw98rpcp8bm0000gn/T/ipykernel_24789/889288480.py", line 3, in <module>
    val = a[4]
IndexError: list index out of range


In [28]:
# If we do not know what kind of error will be raised:
import traceback
try:
    a = [1, 2, 3]
    val = a[4]
except:
    logging.error("The error is %s", traceback.format_exc())

2022-09-17 09:11:59,886 - root - ERROR - The error is Traceback (most recent call last):
  File "/var/folders/wx/x69ggs7s5cgd2gw98rpcp8bm0000gn/T/ipykernel_24789/147431224.py", line 5, in <module>
    val = a[4]
IndexError: list index out of range



## ROTATING FILE HANDLER:
In a large app with a lot of logging messages, you can use a rotating file handler that keeps the file of all the logs small.

In [30]:
from logging.handlers import RotatingFileHandler

In [33]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# roll over after 2KB and keep backup logs app.log.1, app.log.2, etc
handler = RotatingFileHandler('app.log', maxBytes = 2000, backupCount=5)

logger.addHandler(handler)

# The following code exhibits the above
# for _ in range(10000):
#    logger.info('Hello, World!')

In [35]:
from logging.handlers import TimedRotatingFileHandler
import time

In [37]:
# Timed Rotating File Handler:

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# roll over? s for seconds, m for min, h for hour, a weekday, etc
# This is set to seconds of 5, so every 5 seconds, a new file
# will be created and continue logging
handler = TimedRotatingFileHandler('timed_test.log', when='s', interval=5, backupCount=5)

logger.addHandler(handler)

# The following code exhibits the above
for _ in range(6):
    logger.info('Hello, World!')
    time.sleep(5)   # Tell it to sleep for 5 sec

KeyboardInterrupt: 

### python-json-logger on GitHub for logging from many modules
can be pip installed
USAGE - explained on github